### Init model from config.yaml (and checkpoint)

In [ ]:
import os
import yaml

import hydra
import torch
import transformers
from transformers import AutoModel
from composer.models import write_huggingface_pretrained_from_composer_checkpoint
from omegaconf import OmegaConf

from scripts.utils import load_model_from_ckpt_dir_path

In [ ]:
path_to_ckpt_dir = ""  # TODO: Fill in ckpt_dir path
with open(os.path.join(path_to_ckpt_dir, "config.yaml"), "rb") as f:
    config = yaml.safe_load(f)
config = OmegaConf.create(config)

In [ ]:
tokenizer = hydra.utils.instantiate(config.tokenizer)

In [ ]:
ckpt_file = "best-rank0.pt"
use_ema = True
model = load_model_from_ckpt_dir_path(
    path_to_ckpt_dir=path_to_ckpt_dir,
    ckpt_file=ckpt_file,
    load_ema_weights=use_ema,
)

### Test push to hub

In [ ]:
from src.utils import save_pretrained_or_push_to_hub

In [ ]:
save_pretrained_or_push_to_hub(
    model=model,
    tokenizer=tokenizer,
    repo_id="",  # TODO: Set this to local path and `local=True` below to test local saving
    private=False,
    # local=True,  # TODO: Uncomment for testing local saving
)

#### Test round trip

In [ ]:
from transformers import AutoModelForMaskedLM

In [ ]:
test_model = AutoModelForMaskedLM.from_pretrained(
    "",  # TODO: Change to local path from above to test loading from locally saved checkpoint
    trust_remote_code=True,
)

In [ ]:
test_model